<h1>LDA</h1>

参考资料：

[1] Latent Dirichlet Allocation, Blei et al. 2003.

# 0. LDA

<img src="files/pic/lda.png" height="500" width="500">

LDA图模型表示，图片来源[1]。

## 0.0 Generative Process

（1）选择 $N \sim Possion(\zeta)$

（2）选择 $\boldsymbol{\theta} \sim Dir(\boldsymbol{\alpha})$

（3）对于$N$个词中的每个词$w_n$：

> + 选择一个主题 $z_n \sim Multinomial(\boldsymbol{\theta}) $
    
> + 从后验分布$p(w_n|z_n, \beta)$中选择单词$w_n$

$\boldsymbol{z}$服从多项式分布，选择$K$维向量$\boldsymbol{\theta}$为多项式分布的共轭分布Dirichlet分布。

> $p(\boldsymbol{\theta}|\boldsymbol{\alpha}) = 
\frac{\Gamma(\sum_{k=1}^K\alpha_k)}{\prod_{k=1}^K\Gamma({\alpha_k})}
\theta_1^{\alpha_1-1}...\theta_k^{\alpha_k-1}$

给定超参数$\boldsymbol{\alpha}$与$\boldsymbol{\beta}$（$\beta_{ij} = p(w^j=1|z^i=1)$），
主题混合分布$\boldsymbol{\theta}$，
$N$个主题组合$\boldsymbol{z}$以及$N$个单词$\boldsymbol{w}$的联合分布是：

> $p(\boldsymbol{\theta}, \boldsymbol{z}, \boldsymbol{w}
|\boldsymbol{\alpha}, \boldsymbol{\beta}) 
= p(\boldsymbol{\theta}|\boldsymbol{\alpha})
\prod_{n=1}^{N}p(z_n|\boldsymbol{\theta})
p(w_n|z_n,\boldsymbol{\beta})$

通过对$\boldsymbol{\theta}$求积分，对$\boldsymbol{z}$求和，一篇文档$\boldsymbol{w}$的边缘分布为：

> $p(\boldsymbol{w}|\boldsymbol{\alpha},\boldsymbol{\beta}) = 
\int 
p(\boldsymbol{\theta}|\boldsymbol{\alpha})
\left( \prod_{n=1}^{N}\sum_{z_n}p(z_n|\boldsymbol{\theta})
p(w_n|z_n,\boldsymbol{\beta}) 
\right) d\boldsymbol{\theta} $

对文档求积分，得到整个数据集$D$的边缘分布：

>  $p(\boldsymbol{D}|\boldsymbol{\alpha},\boldsymbol{\beta}) = 
\prod_{d=1}^M
\int 
p(\boldsymbol{\theta_d}|\boldsymbol{\alpha})
\left(
\prod_{n=1}^{N_d}\sum_{z_{dn}}p(z_{dn}|\boldsymbol{\theta_d})
p(w_{dn}|z_{dn},\boldsymbol{\beta}) 
\right)
d \boldsymbol{\theta_d} $


## 0.1 Inference

> $p(\boldsymbol{w}|\boldsymbol{\alpha},\boldsymbol{\beta}) = 
\int 
p(\boldsymbol{\theta}|\boldsymbol{\alpha})
\left(
\prod_{n=1}^{N}\sum_{z_n}p(z_n|\boldsymbol{\theta})
p(w_n|z_n,\boldsymbol{\beta})
\right)
d\boldsymbol{\theta} $

> $ = \frac{\Gamma(\sum_{k=1}^K\alpha_k)}{\prod_{k=1}^K\Gamma({\alpha_k})} 
\int \left(\prod_{k=1}^K \theta_k^{\alpha_k-1} \right)
\left(
\prod_{n=1}^{N}\sum_{k=1}^{K}
\prod_{j=1}^{V}
(\theta_k)^{w_n^j}
(\beta_{kj})^{w_n^j}
\right)
d\boldsymbol{\theta}
$

> $ = \frac{\Gamma(\sum_{k=1}^K\alpha_k)}{\prod_{k=1}^K\Gamma({\alpha_k})} 
\int \left(\prod_{k=1}^K \theta_k^{\alpha_k-1} \right)
\left(
\prod_{n=1}^{N}\sum_{k=1}^{K}
\prod_{j=1}^{V}
(\theta_k\beta_{kj})^{\mathbb{I}(w_n=j)}
\right)
d\boldsymbol{\theta}
$

$p(\boldsymbol{w}|\boldsymbol{\alpha},\boldsymbol{\beta})$是难以计算（intractable）的。

## 0.2 Variational Inference

假设$\gamma$、$\phi$分别是隐变量$\theta$以及$z$的变分参数，从而隐变量的平均场变分分布族可表示为，

> $q(\boldsymbol{\theta},\boldsymbol{z}|\boldsymbol{\gamma},\boldsymbol{\phi}) 
= q(\boldsymbol{\theta}|\boldsymbol{\gamma})
\prod_{n=1}^{N}q(z_n|\phi_n)$

根据mean-field变分方法，

> $(\gamma^*,\phi^*) = \arg\min_{(\gamma, \phi)}
\mathbb{KL}[q(\theta,z|\gamma,\phi) || p(\theta,z|w,\alpha,\beta) ]$